# ABSTRACT

Hyperparameters are parameters that are specified prior to running machine learning algorithms that have a large effect on the predictive power of statistical models. Knowledge of the relative importance of a hyperparameter to an algorithm and its range of values is crucial to hyperparameter tuning and creating effective models. To either experts or non-experts, determining hyperparameters that optimize model performance can be a tedious and difficult task. Therefore, we develop a hyperparameter database that allows users to visualize and understand how to choose hyperparameters that maximize the predictive power of their models. 

The database is created by running millions of hyperparameter values, over thousands of public datasets and calculating the individual conditional expectation of every hyperparameter to the quality of a model.                 

We analyze the **effect of hyperparameters** on algorithms such as                                                  
Distributed Random Forest (DRF),                                                                               
Generalized Linear Model (GLM),                                                                                
Gradient Boosting Machine (GBM),                                                                            
Boosting (XGBoost) and several more.                                                                          
Consequently, the database attempts to provide a one-stop platform for data scientists to identify hyperparameters that have the most effect on their models in order to speed up the process of developing effective predictive models. Moreover, the database will also use these public datasets to build models that can predict hyperparameters without search and for visualizing and teaching concepts such as statistical power and bias/variance tradeoff. The raw data will also be publically available for the research community.


In [25]:
# import h2o package and specific estimator 
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

import warnings
warnings.filterwarnings('ignore')

In [26]:
h2o.init(strict_version_check=False) # start h2o

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,55 mins 56 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,"14 days, 13 hours and 1 minute"
H2O cluster name:,H2O_from_python_newzysharma_a1fn1u
H2O cluster total nodes:,1
H2O cluster free memory:,1.987 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [27]:
#importing data to the server
hp = h2o.import_file(path="hour.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [28]:
#Displaying the head
hp.head()

instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,2011-01-01 00:00:00,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
2,2011-01-01 00:00:00,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0,8,32,40
3,2011-01-01 00:00:00,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0,5,27,32
4,2011-01-01 00:00:00,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0,3,10,13
5,2011-01-01 00:00:00,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0,0,1,1
6,2011-01-01 00:00:00,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
7,2011-01-01 00:00:00,1,0,1,6,0,6,0,1,0.22,0.2727,0.8,0,2,0,2
8,2011-01-01 00:00:00,1,0,1,7,0,6,0,1,0.2,0.2576,0.86,0,1,2,3
9,2011-01-01 00:00:00,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0,1,7,8
10,2011-01-01 00:00:00,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0,8,6,14


In [29]:
hp.describe()

Rows:17379
Cols:17




,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
type,int,time,int,int,int,int,int,int,int,int,real,real,real,real,int,int,int
mins,1.0,1293840000000.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.02,0.0,0.0,0.0,0.0,0.0,1.0
mean,8690.0,1325477314552.0461,2.501639910236492,0.5025605615973301,6.5377754761493785,11.546751826917548,0.028770355026181024,3.003682605443351,0.6827205247712756,1.425283387997008,0.4969871684216584,0.47577510213476026,0.6272288394038784,0.1900976063064618,35.67621842453536,153.78686920996606,189.4630876345014
maxs,17379.0,1356912000000.0,4.0,1.0,12.0,23.0,1.0,6.0,1.0,4.0,1.0,1.0,1.0,0.8507,367.0,886.0,977.0
sigma,5017.029499614288,18150225217.779854,1.1069181394480765,0.5000078290910197,3.438775713750168,6.914405095264493,0.16716527638437123,2.005771456110988,0.4654306335238829,0.6393568777542534,0.19255612124972193,0.17185021563535932,0.19292983406291514,0.1223402285727905,49.30503038705309,151.35728591258314,181.38759909186476
zeros,0,0,0,8645,0,726,16879,2502,5514,0,0,2,22,2180,1581,24,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1.0,2011-01-01 00:00:00,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0,3.0,13.0,16.0
1,2.0,2011-01-01 00:00:00,1.0,0.0,1.0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.8,0.0,8.0,32.0,40.0
2,3.0,2011-01-01 00:00:00,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.8,0.0,5.0,27.0,32.0


In [55]:
# Functions

def alphabet(n):
  alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
  str=''
  r=len(alpha)-1   
  while len(str)<n:
    i=random.randint(0,r)
    str+=alpha[i]   
  return str
  
  
def set_meta_data(analysis,run_id,server,data,test,model_path,target,run_time,classification,scale,model,balance,balance_threshold,name,path,nthreads,min_mem_size):
  m_data={}
  m_data['start_time'] = time.time()
  m_data['target']=target
  m_data['server_path']=server
  m_data['data_path']=data 
  m_data['test_path']=test
  m_data['max_models']=model
  m_data['run_time']=run_time
  m_data['run_id'] =run_id
  m_data['scale']=scale
  m_data['classification']=classification
  m_data['scale']=False
  m_data['model_path']=model_path
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()
  m_data['execution_time'] = 0.0
  m_data['run_path'] =path
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data


def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
  
  
def stackedensemble(mod):
    coef_norm=None
    try:
      metalearner = h2o.get_model(mod.metalearner()['name'])
      coef_norm=metalearner.coef_norm()
    except:
      pass        
    return coef_norm

def stackedensemble_df(df):
    bm_algo={ 'GBM': None,'GLM': None,'DRF': None,'XRT': None,'Dee': None}
    for index, row in df.iterrows():
      if len(row['model_id'])>3:
        key=row['model_id'][0:3]
        if key in bm_algo:
          if bm_algo[key] is None:
                bm_algo[key]=row['model_id']
    bm=list(bm_algo.values()) 
    bm=list(filter(None.__ne__, bm))             
    return bm

def se_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['auc']=modl.auc()   
    d['roc']=modl.roc()
    d['mse']=modl.mse()   
    d['null_degrees_of_freedom']=modl.null_degrees_of_freedom()
    d['null_deviance']=modl.null_deviance()
    d['residual_degrees_of_freedom']=modl.residual_degrees_of_freedom()   
    d['residual_deviance']=modl.residual_deviance()
    d['rmse']=modl.rmse()
    return d

def get_model_by_algo(algo,models_dict):
    mod=None
    mod_id=None    
    for m in list(models_dict.keys()):
        if m[0:3]==algo:
            mod_id=m
            mod=h2o.get_model(m)      
    return mod,mod_id     
    
    
def gbm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def dl_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def drf_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
def xrt_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
    
def glm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['coef']=modl.coef()  
    d['coef_norm']=modl.coef_norm()      
    return d
    
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:
      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d
    
def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return


def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
    

def check_all_variables(df,dct,y=None):     
    targ=list(dct.keys())     
    for key, val in df.types.items():
        if key in targ:
          if dct[key] not in ['real','int','enum']:                      
            targ.remove(key)  
    for key, val in df.types.items():
        if key in targ:            
          if dct[key] != val:
            print('convert ',key,' ',dct[key],' ',val)
            if dct[key]=='enum':
                try:
                  df[key] = df[key].asfactor() 
                except:
                  targ.remove(key)                 
            if dct[key]=='int': 
                try:                
                  df[key] = df[key].asnumeric() 
                except:
                  targ.remove(key)                  
            if dct[key]=='real':
                try:                
                  df[key] = df[key].asnumeric()  
                except:
                  targ.remove(key)                  
    if y is None:
      y=df.columns[-1] 
    if y in targ:
      targ.remove(y)
    else:
      y=targ.pop()            
    return targ    
    
def predictions(mod,data,run_id):
    test = h2o.import_file(data)
    mod_perf=mod_best.model_performance(test)
              
    stats_test={}
    stats_test=model_performance_stats(mod_perf)

    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 

    try:    
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass

    predictions = mod_best.predict(test)
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return

def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    
#  End Functions

In [56]:
all_variables=None

# Model with 500 seconds

In [32]:
# Assume the following are passed by the user from the web interface

'''
Need a user id and project id?

'''
target='cnt' 
data_file='hour.csv'
run_time=500
run_id='SOME_ID_20180617_221528' # Just some arbitrary ID
server_path='/Users/newzysharma/Desktop/Desktop/Machine_Learning/Project/RunTime_500Secondst'
classification=False
scale=False
max_models=None
balance_y=False # balance_classes=balance_y
balance_threshold=0.2
project ="HyperparameterDB_Project"  # project_name = project

In [33]:
# assign target and inputs for logistic regression
y = target
X = [name for name in hp.columns if name != y]
print(y)
print(X)

cnt
['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered']


In [34]:
# determine column types
ints, reals, enums = [], [], []
for key, val in hp.types.items():
    if key in X:
        if val == 'enum':
            enums.append(key)
        elif val == 'int':
            ints.append(key)            
        else: 
            reals.append(key)

print(ints)
print(enums)
print(reals)

['instant', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'casual', 'registered']
[]
['dteday', 'temp', 'atemp', 'hum', 'windspeed']


In [35]:
# impute missing values
_ = hp[reals].impute(method='mean')
_ = hp[ints].impute(method='median')

if scale:
    hp[reals] = df[reals].scale()
    hp[ints] = df[ints].scale()

In [36]:
# # set target to factor for classification by default or if user specifies classification
# if classification:
#     [y] = hp[y].asfactor()

In [37]:
hp[y].levels()

[]


## Cross-validate rather than take a test training split with 500 seconds

In [38]:
# automl
# runs for run_time seconds then builds a stacked ensemble

aml = H2OAutoML(max_runtime_secs=run_time,project_name = project) # init automl, run for 500 seconds
aml.train(x=X,  
           y=y,
           training_frame=hp)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Leaderboard

In [39]:
# view leaderboard
lb = aml.leaderboard
lb.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20190415_114652,18.2238,4.26893,18.2238,2.65155,0.0689224
XGBoost_2_AutoML_20190415_124157,20.8715,4.56853,20.8715,2.6007,0.0483929
XGBoost_1_AutoML_20190415_124157,28.4471,5.33359,28.4471,3.13775,0.0651591
XGBoost_3_AutoML_20190415_124157,35.0583,5.921,35.0583,3.93716,0.113491
DRF_1_AutoML_20190415_124157,138.235,11.7573,138.235,6.86539,0.113663
GLM_grid_1_AutoML_20190415_124157_model_1,32707.5,180.852,32707.5,141.957,1.56795


In [40]:
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20190415_114652


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 2.814884061845404
RMSE: 1.677761622473647
MAE: 1.1759801045530882
RMSLE: 0.03735902097772964
Mean Residual Deviance: 2.814884061845404

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 18.223768662785524
RMSE: 4.268930622859257
MAE: 2.651545887114065
RMSLE: 0.06892242390465715
Mean Residual Deviance: 18.223768662785524
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,2.6515534,0.0505039,2.5708938,2.6065495,2.636067,2.6639316,2.7803247
mean_residual_deviance,18.223953,1.4051609,15.396717,17.268597,18.9354,18.083717,21.435335
mse,18.223953,1.4051609,15.396717,17.268597,18.9354,18.083717,21.435335
r2,0.9994459,0.0000441,0.9995326,0.9994791,0.9994212,0.9994514,0.9993449
residual_deviance,18.223953,1.4051609,15.396717,17.268597,18.9354,18.083717,21.435335
rmse,4.262645,0.1640311,3.923865,4.15555,4.3514824,4.2524953,4.629831
rmsle,0.0681753,0.0071522,0.084033,0.0670370,0.0661790,0.0711374,0.0524904


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-04-15 11:52:09,5 min 17.106 sec,0.0,261.9286476,188.9630876,68606.6164192
,2019-04-15 11:52:09,5 min 17.590 sec,5.0,204.2654402,146.6653762,41724.3700544
,2019-04-15 11:52:10,5 min 17.948 sec,10.0,160.3255850,113.8386434,25704.2931947
,2019-04-15 11:52:10,5 min 18.325 sec,15.0,125.3110979,88.3999035,15702.8712543
,2019-04-15 11:52:10,5 min 18.671 sec,20.0,98.1829369,68.6941081,9639.8890984
---,---,---,---,---,---,---
,2019-04-15 11:53:25,6 min 33.742 sec,405.0,1.7404701,1.2150934,3.0292363
,2019-04-15 11:53:26,6 min 34.692 sec,410.0,1.7153898,1.1989803,2.9425621
,2019-04-15 11:53:27,6 min 35.648 sec,415.0,1.7010231,1.1902921,2.8934797
,2019-04-15 11:53:28,6 min 36.661 sec,420.0,1.6870757,1.1813391,2.8462244



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
registered,1867817856.0000000,1.0,0.5348379
casual,953776896.0000000,0.5106370,0.2731080
hr,400109056.0000000,0.2142120,0.1145687
workingday,95300912.0000000,0.0510226,0.0272888
dteday,63556376.0000000,0.0340271,0.0181990
instant,44114180.0000000,0.0236180,0.0126318
weekday,27750280.0000000,0.0148571,0.0079461
temp,12800581.0000000,0.0068532,0.0036654
atemp,9081057.0,0.0048619,0.0026003
hum,4654595.5,0.0024920,0.0013328


In [13]:
aml.leader.algo

'xgboost'

## Ensemble Exploration

In [49]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,XGBoost_1_AutoML_20190415_114652,18.223769,4.268931,18.223769,2.651546,0.068922
1,XGBoost_2_AutoML_20190415_124157,20.871452,4.568528,20.871452,2.600701,0.048393
2,XGBoost_1_AutoML_20190415_124157,28.447149,5.333587,28.447149,3.137746,0.065159
3,XGBoost_3_AutoML_20190415_124157,35.058291,5.921004,35.058291,3.937160,0.113491
4,DRF_1_AutoML_20190415_124157,138.234686,11.757325,138.234686,6.865390,0.113663
5,GLM_grid_1_AutoML_20190415_124157_model_1,32707.524871,180.852218,32707.524871,141.956925,1.567950


## Getting Models

### Parameters for XGBoost_1_AutoML_20190415_114652

In [51]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [52]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'XGBoost_1_AutoML_20190415_114652',
   'type': 'Key<Model>',
   'URL': '/3/Models/XGBoost_1_AutoML_20190415_114652'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'fold_assignment': {'default': 'AUTO', 'actual': 'Modulo'},
 'fold_column': {'defau

In [58]:
hyper_parameter_xgboost = mod_best.params
n=run_id+'XGBoost_1_AutoML_20190415_114652.json'
dict_to_json(hyper_parameter,n)

NameError: name 'hyper_parameter' is not defined

#### generating different models for json file

In [46]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[2])

In [47]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'XGBoost_1_AutoML_20190415_124157',
   'type': 'Key<Model>',
   'URL': '/3/Models/XGBoost_1_AutoML_20190415_124157'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour1.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour1.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'fold_assignment': {'default': 'AUTO', 'actual': 'Modulo'},
 'fold_column': {'def

In [42]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20190414_221634
No model summary for this model


ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 228.05154670500258
RMSE: 15.101375656045464
MAE: 11.63634317115643
RMSLE: 0.49716685876783684
R^2: 0.993068251012167
Mean Residual Deviance: 228.05154670500258
Null degrees of freedom: 17378
Residual degrees of freedom: 17375
Null deviance: 571761591.0707175
Residual deviance: 3963307.83018624
AIC: 143689.9918584112

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 404.55206573805515
RMSE: 20.113479702380072
MAE: 15.286993185084869
RMSLE: 0.5657017501133251
R^2: 0.9877034231395371
Mean Residual Deviance: 404.55206573805515
Null degrees of freedom: 17378
Residual degrees of freedom: 17375
Null deviance: 571800073.8579907
Residual deviance: 7030710.35046166
AIC: 153651.78688213223
<bound method ModelBase.coef_nor

In [44]:
#%matplotlib inline
#mod_best.std_coef_plot()

### Parameters for StackedEnsemble_AllModels_AutoML_20190408_134347

In [47]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[4])

In [48]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'DRF_1_AutoML_20190414_221634',
   'type': 'Key<Model>',
   'URL': '/3/Models/DRF_1_AutoML_20190414_221634'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 0},
 'fold_assignment': {'default': 'AUTO',

In [49]:
hy_parameter_stack = mod_best.params
n=run_id+'StackedEnsemble_AllModels_AutoML_20190408_171007.json'
dict_to_json(hy_parameter,n)

### BUG:  I am trying to get Parameters for model 6(GBM_1_AutoML_20190408_134347) but i am getting some other model but i am getting GBM_4_AutoML_20190408_171007

In [105]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[6])

In [106]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_4_AutoML_20190408_171007',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_4_AutoML_20190408_171007'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 5},
 'fold_assignment': {'default': 'AUTO',

# Model with runtime 1000 seconds

In [57]:
# Assume the following are passed by the user from the web interface

'''
Need a user id and project id?

'''
target='cnt' 
data_file='hour.csv'
run_time=1000
run_id='SOME_ID_20180617_221529' # Just some arbitrary ID
server_path='Users/newzysharma/Desktop/Desktop/Machine_Learning/Project'
classification=False
scale=False
max_models=None
balance_y=False # balance_classes=balance_y
balance_threshold=0.2
project ="HyperparameterDB_Project"  # project_name = project

In [58]:
# Use local data file or download from some type of bucket
import os

data_path=os.path.join(server_path,data_file)
data_path

'Users/newzysharma/Desktop/Desktop/Machine_Learning/Project/hour.csv'

In [59]:
# assign target and inputs for logistic regression
y = target
X = [name for name in hp.columns if name != y]
print(y)
print(X)

cnt
['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered']


In [60]:
# determine column types
ints, reals, enums = [], [], []
for key, val in hp.types.items():
    if key in X:
        if val == 'enum':
            enums.append(key)
        elif val == 'int':
            ints.append(key)            
        else: 
            reals.append(key)

print(ints)
print(enums)
print(reals)

['instant', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'casual', 'registered']
[]
['dteday', 'temp', 'atemp', 'hum', 'windspeed']


In [61]:
# impute missing values
_ = hp[reals].impute(method='mean')
_ = hp[ints].impute(method='median')

if scale:
    hp[reals] = hp[reals].scale()
    hp[ints] = hp[ints].scale()

In [20]:
# set target to factor for classification by default or if user specifies classification
#if classification:
    #hp[y] = hp[y].asfactor()

In [31]:
hp[y].levels()

[]

In [62]:
if classification:
    class_percentage = y_balance=df[y].mean()[0]/(df[y].max()-df[y].min())
    if class_percentage < balance_threshold:
        balance_y=True
        

print(run_time)
type(run_time)

1000


int


## Cross-validate rather than take a test training split with 1000 seconds

In [63]:
# automl
# runs for run_time seconds then builds a stacked ensemble

aml = H2OAutoML(max_runtime_secs=run_time,project_name = project) # init automl, run for 1000 seconds
aml.train(x=X,  
           y=y,
           training_frame=hp)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Leaderboard

In [64]:
# view leaderboard
lb = aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_2_AutoML_20190408_171007,10.3989,3.22474,10.3989,1.88964,0.0353651
GBM_1_AutoML_20190408_171007,13.2483,3.63982,13.2483,2.30598,0.0573338
XGBoost_1_AutoML_20190408_171007,14.936,3.86471,14.936,2.25298,0.0426858
GBM_3_AutoML_20190408_171007,22.3503,4.72761,22.3503,2.93989,0.0688736
StackedEnsemble_AllModels_AutoML_20190408_171007,23.1046,4.80673,23.1046,3.22004,0.180671
XGBoost_3_AutoML_20190408_171007,23.4324,4.84071,23.4324,3.16966,0.0785482
GBM_4_AutoML_20190408_171007,25.8815,5.08739,25.8815,2.98719,nan
XGBoost_2_AutoML_20190408_171007,31.3904,5.60271,31.3904,3.26648,0.0692559
GLM_grid_1_AutoML_20190408_171007_model_1,38.387,6.19573,38.387,4.45197,nan
DRF_1_AutoML_20190408_171007,44.9979,6.70804,44.9979,3.64084,0.0762501


In [65]:
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_2_AutoML_20190408_171007


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 2.7545139005515074
RMSE: 1.659672829370749
MAE: 1.1420248950973013
RMSLE: 0.026260655890205944
Mean Residual Deviance: 2.7545139005515074

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 10.398929973581447
RMSE: 3.2247371944984056
MAE: 1.8896396840192615
RMSLE: 0.03536506290104928
Mean Residual Deviance: 10.398929973581447
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,1.8896418,0.0331531,1.8888159,1.8440878,1.955546,1.8330745,1.9266849
mean_residual_deviance,10.398917,1.1037041,9.484616,8.705696,13.301646,10.319401,10.1832285
mse,10.398917,1.1037041,9.484616,8.705696,13.301646,10.319401,10.1832285
r2,0.9996837,0.0000345,0.9997121,0.9997374,0.9995934,0.9996870,0.9996888
residual_deviance,10.398917,1.1037041,9.484616,8.705696,13.301646,10.319401,10.1832285
rmse,3.2161787,0.1659996,3.0797105,2.9505417,3.6471422,3.2123823,3.191117
rmsle,0.0353167,0.0013090,0.0337463,0.0375869,0.0355834,0.0327149,0.0369519


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-04-08 17:25:27,16.816 sec,0.0,181.3823804,142.3998489,32899.5679309
,2019-04-08 17:25:27,16.882 sec,5.0,107.4239264,84.2400360,11539.8999535
,2019-04-08 17:25:27,16.935 sec,10.0,63.8940838,50.0233089,4082.4539467
,2019-04-08 17:25:27,16.985 sec,15.0,38.4509700,30.0349704,1478.4770913
,2019-04-08 17:25:27,17.034 sec,20.0,25.6530386,19.7348606,658.0783895
---,---,---,---,---,---,---
,2019-04-08 17:25:30,19.344 sec,270.0,1.7050142,1.1700358,2.9070734
,2019-04-08 17:25:30,19.393 sec,275.0,1.6909674,1.1622558,2.8593708
,2019-04-08 17:25:30,19.451 sec,280.0,1.6770692,1.1544659,2.8125612
,2019-04-08 17:25:30,19.520 sec,285.0,1.6617166,1.1441284,2.7613021



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
registered,2568583680.0000000,1.0,0.9182406
casual,153480320.0000000,0.0597529,0.0548675
hr,43721364.0000000,0.0170216,0.0156299
instant,9335018.0,0.0036343,0.0033372
workingday,7972549.0,0.0031039,0.0028501
dteday,5198927.5,0.0020240,0.0018586
temp,3354083.5,0.0013058,0.0011990
weekday,2513148.2500000,0.0009784,0.0008984
atemp,2094532.2500000,0.0008154,0.0007488
hum,538358.8125000,0.0002096,0.0001925


In [66]:
aml.leader.algo

'gbm'

## Ensemble Exploration

In [67]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,GBM_2_AutoML_20190408_171007,10.398930,3.224737,10.398930,1.889640,0.035365
1,GBM_1_AutoML_20190408_171007,13.248308,3.639823,13.248308,2.305975,0.057334
2,XGBoost_1_AutoML_20190408_171007,14.936004,3.864713,14.936004,2.252979,0.042686
3,GBM_3_AutoML_20190408_171007,22.350299,4.727610,22.350299,2.939890,0.068874
4,StackedEnsemble_AllModels_AutoML_20190408_171007,23.104649,4.806730,23.104649,3.220036,0.180671
5,XGBoost_3_AutoML_20190408_171007,23.432444,4.840707,23.432444,3.169660,0.078548
6,GBM_4_AutoML_20190408_171007,25.881513,5.087388,25.881513,2.987189,NaN
7,XGBoost_2_AutoML_20190408_171007,31.390413,5.602715,31.390413,3.266475,0.069256
8,GLM_grid_1_AutoML_20190408_171007_model_1,38.387049,6.195728,38.387049,4.451969,NaN
9,DRF_1_AutoML_20190408_171007,44.997852,6.708044,44.997852,3.640842,0.076250


##### As per the rmse metric that "The smaller the RMSE value, the better the model".
##### As per the mse metric that "The smaller the MSE value, the better the model".
So, In our case GBM_2_AutoML_20190408_171007 has the smallest rmse = 3.224737 and mse = 10.398930 that is why it is our best model.

## Getting Models

### Parameters for GBM_2_AutoML_20190408_171007

In [69]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [70]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_2_AutoML_20190408_171007',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_2_AutoML_20190408_171007'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 5},
 'fold_assignment': {'default': 'AUTO',

### Parameters for XGBoost_1_AutoML_20190408_171007

In [82]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[2])

In [83]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'XGBoost_1_AutoML_20190408_171007',
   'type': 'Key<Model>',
   'URL': '/3/Models/XGBoost_1_AutoML_20190408_171007'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'fold_assignment': {'default': 'AUTO', 'actual': 'Modulo'},
 'fold_column': {'defau

### Parameters for StackedEnsemble_AllModels_AutoML_20190408_171007

In [86]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[4])

In [87]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'StackedEnsemble_AllModels_AutoML_20190408_171007',
   'type': 'Key<Model>',
   'URL': '/3/Models/StackedEnsemble_AllModels_AutoML_20190408_171007'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'response_column': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ColSpecifierV3',
    'schema_type': 'VecSpecifier'},
   'column_name': 'cnt',
   'is_member_of_frames': None}},
 'validation_frame': {'default': None, 'actual': None},
 'blending_frame': {'default': None, 'actual': None},
 'base_models': {'default': [],
  'actual': [{'__meta': {'schema_version': 3,
     'schema_name': 'ModelKey

## Model with runtime 1350 seconds

In [128]:
# Assume the following are passed by the user from the web interface

'''
Need a user id and project id?

'''
target='cnt' 
data_file='hour.csv'
run_time=1350
run_id='SOME_ID_20180617_221530' # Just some arbitrary ID
server_path='Users/newzysharma/Desktop/Desktop/Machine_Learning/Project'
classification=False
scale=False
max_models=None
balance_y=False # balance_classes=balance_y
balance_threshold=0.2
project ="HyperparameterDB_Project"  # project_name = project

In [109]:
# Use local data file or download from some type of bucket
import os

data_path=os.path.join(server_path,data_file)
data_path

'Users/newzysharma/Desktop/Desktop/Machine_Learning/Project/hour.csv'

In [110]:
# assign target and inputs for logistic regression
y = target
X = [name for name in hp.columns if name != y]
print(y)
print(X)

cnt
['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered']


In [111]:
# determine column types
ints, reals, enums = [], [], []
for key, val in hp.types.items():
    if key in X:
        if val == 'enum':
            enums.append(key)
        elif val == 'int':
            ints.append(key)            
        else: 
            reals.append(key)

print(ints)
print(enums)
print(reals)

['instant', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'casual', 'registered']
[]
['dteday', 'temp', 'atemp', 'hum', 'windspeed']


In [112]:
# impute missing values
_ = hp[reals].impute(method='mean')
_ = hp[ints].impute(method='median')

if scale:
    hp[reals] = hp[reals].scale()
    hp[ints] = hp[ints].scale()

In [113]:
if classification:
    class_percentage = y_balance=df[y].mean()[0]/(df[y].max()-df[y].min())
    if class_percentage < balance_threshold:
        balance_y=True
        

print(run_time)
type(run_time)

1350


int

## Cross-validate rather than take a test training split with 1350 seconds

In [114]:
# automl
# runs for run_time seconds then builds a stacked ensemble

aml = H2OAutoML(max_runtime_secs=run_time,project_name = project) # init automl, run for 1350 seconds
aml.train(x=X,  
           y=y,
           training_frame=hp)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Leaderboard

In [115]:
# view leaderboard
lb = aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_2_AutoML_20190408_171007,10.3989,3.22474,10.3989,1.88964,0.0353651
GBM_1_AutoML_20190408_171007,13.2483,3.63982,13.2483,2.30598,0.0573338
GBM_4_AutoML_20190408_190707,14.2118,3.76985,14.2118,2.15209,0.0367956
XGBoost_1_AutoML_20190408_171007,14.936,3.86471,14.936,2.25298,0.0426858
GBM_3_AutoML_20190408_190707,15.2039,3.89922,15.2039,2.27726,0.0438891
GBM_1_AutoML_20190408_190707,19.1514,4.37623,19.1514,2.70842,nan
XGBoost_2_AutoML_20190408_190707,19.5511,4.42167,19.5511,2.52855,0.050435
XGBoost_1_AutoML_20190408_190707,20.9605,4.57826,20.9605,2.65315,0.0493793
GBM_2_AutoML_20190408_190707,22.3321,4.72569,22.3321,2.82382,nan
GBM_3_AutoML_20190408_171007,22.3503,4.72761,22.3503,2.93989,0.0688736


In [116]:
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_2_AutoML_20190408_171007


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 2.7545139005515074
RMSE: 1.659672829370749
MAE: 1.1420248950973013
RMSLE: 0.026260655890205944
Mean Residual Deviance: 2.7545139005515074

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 10.398929973581447
RMSE: 3.2247371944984056
MAE: 1.8896396840192615
RMSLE: 0.03536506290104928
Mean Residual Deviance: 10.398929973581447
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,1.8896418,0.0331531,1.8888159,1.8440878,1.955546,1.8330745,1.9266849
mean_residual_deviance,10.398917,1.1037041,9.484616,8.705696,13.301646,10.319401,10.1832285
mse,10.398917,1.1037041,9.484616,8.705696,13.301646,10.319401,10.1832285
r2,0.9996837,0.0000345,0.9997121,0.9997374,0.9995934,0.9996870,0.9996888
residual_deviance,10.398917,1.1037041,9.484616,8.705696,13.301646,10.319401,10.1832285
rmse,3.2161787,0.1659996,3.0797105,2.9505417,3.6471422,3.2123823,3.191117
rmsle,0.0353167,0.0013090,0.0337463,0.0375869,0.0355834,0.0327149,0.0369519


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-04-08 17:25:27,16.816 sec,0.0,181.3823804,142.3998489,32899.5679309
,2019-04-08 17:25:27,16.882 sec,5.0,107.4239264,84.2400360,11539.8999535
,2019-04-08 17:25:27,16.935 sec,10.0,63.8940838,50.0233089,4082.4539467
,2019-04-08 17:25:27,16.985 sec,15.0,38.4509700,30.0349704,1478.4770913
,2019-04-08 17:25:27,17.034 sec,20.0,25.6530386,19.7348606,658.0783895
---,---,---,---,---,---,---
,2019-04-08 17:25:30,19.344 sec,270.0,1.7050142,1.1700358,2.9070734
,2019-04-08 17:25:30,19.393 sec,275.0,1.6909674,1.1622558,2.8593708
,2019-04-08 17:25:30,19.451 sec,280.0,1.6770692,1.1544659,2.8125612
,2019-04-08 17:25:30,19.520 sec,285.0,1.6617166,1.1441284,2.7613021



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
registered,2568583680.0000000,1.0,0.9182406
casual,153480320.0000000,0.0597529,0.0548675
hr,43721364.0000000,0.0170216,0.0156299
instant,9335018.0,0.0036343,0.0033372
workingday,7972549.0,0.0031039,0.0028501
dteday,5198927.5,0.0020240,0.0018586
temp,3354083.5,0.0013058,0.0011990
weekday,2513148.2500000,0.0009784,0.0008984
atemp,2094532.2500000,0.0008154,0.0007488
hum,538358.8125000,0.0002096,0.0001925


In [118]:
aml.leader.algo

'gbm'

## Ensemble Exploration

In [119]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,GBM_2_AutoML_20190408_171007,10.398930,3.224737,10.398930,1.889640,0.035365
1,GBM_1_AutoML_20190408_171007,13.248308,3.639823,13.248308,2.305975,0.057334
2,GBM_4_AutoML_20190408_190707,14.211758,3.769849,14.211758,2.152092,0.036796
3,XGBoost_1_AutoML_20190408_171007,14.936004,3.864713,14.936004,2.252979,0.042686
4,GBM_3_AutoML_20190408_190707,15.203899,3.899218,15.203899,2.277258,0.043889
5,GBM_1_AutoML_20190408_190707,19.151410,4.376232,19.151410,2.708416,NaN
6,XGBoost_2_AutoML_20190408_190707,19.551146,4.421668,19.551146,2.528554,0.050435
7,XGBoost_1_AutoML_20190408_190707,20.960508,4.578265,20.960508,2.653145,0.049379
8,GBM_2_AutoML_20190408_190707,22.332141,4.725689,22.332141,2.823816,NaN
9,GBM_3_AutoML_20190408_171007,22.350299,4.727610,22.350299,2.939890,0.068874


## Getting models

### Parameters for GBM_2_AutoML_20190408_171007

In [121]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [122]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_2_AutoML_20190408_171007',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_2_AutoML_20190408_171007'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 5},
 'fold_assignment': {'default': 'AUTO',

### Parameters for XGBoost_1_AutoML_20190408_171007

In [124]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[3])

In [125]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'XGBoost_1_AutoML_20190408_171007',
   'type': 'Key<Model>',
   'URL': '/3/Models/XGBoost_1_AutoML_20190408_171007'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'fold_assignment': {'default': 'AUTO', 'actual': 'Modulo'},
 'fold_column': {'defau

### Parameters for DRF_1_AutoML_20190408_171007

In [126]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[20])

In [127]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'DRF_1_AutoML_20190408_171007',
   'type': 'Key<Model>',
   'URL': '/3/Models/DRF_1_AutoML_20190408_171007'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 0},
 'fold_assignment': {'default': 'AUTO',

## Model with runtime 1500 seconds

In [134]:
# Assume the following are passed by the user from the web interface

'''
Need a user id and project id?

'''
target='cnt' 
data_file='hour.csv'
run_time=1500
run_id='SOME_ID_20180617_221531' # Just some arbitrary ID
server_path='Users/newzysharma/Desktop/Desktop/Machine_Learning/Project'
classification=False
scale=False
max_models=None
balance_y=False # balance_classes=balance_y
balance_threshold=0.2
project ="HyperparameterDB_Project"  # project_name = project

In [135]:
# Use local data file or download from some type of bucket
import os

data_path=os.path.join(server_path,data_file)
data_path

'Users/newzysharma/Desktop/Desktop/Machine_Learning/Project/hour.csv'

In [136]:
# assign target and inputs for logistic regression
y = target
X = [name for name in hp.columns if name != y]
print(y)
print(X)

cnt
['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered']


In [137]:
# impute missing values
_ = hp[reals].impute(method='mean')
_ = hp[ints].impute(method='median')

if scale:
    hp[reals] = hp[reals].scale()
    hp[ints] = hp[ints].scale()

In [138]:
if classification:
    class_percentage = y_balance=df[y].mean()[0]/(df[y].max()-df[y].min())
    if class_percentage < balance_threshold:
        balance_y=True
        

print(run_time)
type(run_time)

1500


int

## Cross-validate rather than take a test training split with 1500 seconds

In [139]:
# automl
# runs for run_time seconds then builds a stacked ensemble

aml = H2OAutoML(max_runtime_secs=run_time,project_name = project) # init automl, run for 1500 seconds
aml.train(x=X,  
           y=y,
           training_frame=hp)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Leaderboard

In [140]:
# view leaderboard
lb = aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_2_AutoML_20190408_171007,10.3989,3.22474,10.3989,1.88964,0.0353651
GBM_2_AutoML_20190408_194251,12.7928,3.57671,12.7928,2.13437,0.0450689
GBM_1_AutoML_20190408_171007,13.2483,3.63982,13.2483,2.30598,0.0573338
GBM_4_AutoML_20190408_190707,14.2118,3.76985,14.2118,2.15209,0.0367956
GBM_3_AutoML_20190408_194251,14.2999,3.78152,14.2999,2.17271,0.0358335
XGBoost_1_AutoML_20190408_171007,14.936,3.86471,14.936,2.25298,0.0426858
GBM_3_AutoML_20190408_190707,15.2039,3.89922,15.2039,2.27726,0.0438891
XGBoost_3_AutoML_20190408_194251,15.9838,3.99797,15.9838,2.48441,nan
GBM_1_AutoML_20190408_190707,19.1514,4.37623,19.1514,2.70842,nan
XGBoost_2_AutoML_20190408_190707,19.5511,4.42167,19.5511,2.52855,0.050435


In [141]:
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_2_AutoML_20190408_171007


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 2.7545139005515074
RMSE: 1.659672829370749
MAE: 1.1420248950973013
RMSLE: 0.026260655890205944
Mean Residual Deviance: 2.7545139005515074

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 10.398929973581447
RMSE: 3.2247371944984056
MAE: 1.8896396840192615
RMSLE: 0.03536506290104928
Mean Residual Deviance: 10.398929973581447
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,1.8896418,0.0331531,1.8888159,1.8440878,1.955546,1.8330745,1.9266849
mean_residual_deviance,10.398917,1.1037041,9.484616,8.705696,13.301646,10.319401,10.1832285
mse,10.398917,1.1037041,9.484616,8.705696,13.301646,10.319401,10.1832285
r2,0.9996837,0.0000345,0.9997121,0.9997374,0.9995934,0.9996870,0.9996888
residual_deviance,10.398917,1.1037041,9.484616,8.705696,13.301646,10.319401,10.1832285
rmse,3.2161787,0.1659996,3.0797105,2.9505417,3.6471422,3.2123823,3.191117
rmsle,0.0353167,0.0013090,0.0337463,0.0375869,0.0355834,0.0327149,0.0369519


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-04-08 17:25:27,16.816 sec,0.0,181.3823804,142.3998489,32899.5679309
,2019-04-08 17:25:27,16.882 sec,5.0,107.4239264,84.2400360,11539.8999535
,2019-04-08 17:25:27,16.935 sec,10.0,63.8940838,50.0233089,4082.4539467
,2019-04-08 17:25:27,16.985 sec,15.0,38.4509700,30.0349704,1478.4770913
,2019-04-08 17:25:27,17.034 sec,20.0,25.6530386,19.7348606,658.0783895
---,---,---,---,---,---,---
,2019-04-08 17:25:30,19.344 sec,270.0,1.7050142,1.1700358,2.9070734
,2019-04-08 17:25:30,19.393 sec,275.0,1.6909674,1.1622558,2.8593708
,2019-04-08 17:25:30,19.451 sec,280.0,1.6770692,1.1544659,2.8125612
,2019-04-08 17:25:30,19.520 sec,285.0,1.6617166,1.1441284,2.7613021



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
registered,2568583680.0000000,1.0,0.9182406
casual,153480320.0000000,0.0597529,0.0548675
hr,43721364.0000000,0.0170216,0.0156299
instant,9335018.0,0.0036343,0.0033372
workingday,7972549.0,0.0031039,0.0028501
dteday,5198927.5,0.0020240,0.0018586
temp,3354083.5,0.0013058,0.0011990
weekday,2513148.2500000,0.0009784,0.0008984
atemp,2094532.2500000,0.0008154,0.0007488
hum,538358.8125000,0.0002096,0.0001925


In [142]:
aml.leader.algo

'gbm'

## Ensemble Exploration

In [143]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,GBM_2_AutoML_20190408_171007,10.398930,3.224737,10.398930,1.889640,0.035365
1,GBM_2_AutoML_20190408_194251,12.792848,3.576709,12.792848,2.134374,0.045069
2,GBM_1_AutoML_20190408_171007,13.248308,3.639823,13.248308,2.305975,0.057334
3,GBM_4_AutoML_20190408_190707,14.211758,3.769849,14.211758,2.152092,0.036796
4,GBM_3_AutoML_20190408_194251,14.299879,3.781518,14.299879,2.172706,0.035834
5,XGBoost_1_AutoML_20190408_171007,14.936004,3.864713,14.936004,2.252979,0.042686
6,GBM_3_AutoML_20190408_190707,15.203899,3.899218,15.203899,2.277258,0.043889
7,XGBoost_3_AutoML_20190408_194251,15.983771,3.997971,15.983771,2.484415,NaN
8,GBM_1_AutoML_20190408_190707,19.151410,4.376232,19.151410,2.708416,NaN
9,XGBoost_2_AutoML_20190408_190707,19.551146,4.421668,19.551146,2.528554,0.050435


## Getting Models

### Parameters for GBM_2_AutoML_20190408_171007

In [144]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [145]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_2_AutoML_20190408_171007',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_2_AutoML_20190408_171007'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 5},
 'fold_assignment': {'default': 'AUTO',

### Parameters for XGBoost_1_AutoML_20190408_171007	

In [146]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[5])

In [147]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'XGBoost_1_AutoML_20190408_171007',
   'type': 'Key<Model>',
   'URL': '/3/Models/XGBoost_1_AutoML_20190408_171007'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'fold_assignment': {'default': 'AUTO', 'actual': 'Modulo'},
 'fold_column': {'defau

### Parameters for DRF_1_AutoML_20190408_171007

In [148]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[29])

In [149]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'DRF_1_AutoML_20190408_171007',
   'type': 'Key<Model>',
   'URL': '/3/Models/DRF_1_AutoML_20190408_171007'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 0},
 'fold_assignment': {'default': 'AUTO',

## Model with runtime 1850 seconds

In [150]:
# Assume the following are passed by the user from the web interface

'''
Need a user id and project id?

'''
target='cnt' 
data_file='hour.csv'
run_time=1850
run_id='SOME_ID_20180617_221531' # Just some arbitrary ID
server_path='Users/newzysharma/Desktop/Desktop/Machine_Learning/Project'
classification=False
scale=False
max_models=None
balance_y=False # balance_classes=balance_y
balance_threshold=0.2
project ="HyperparameterDB_Project"  # project_name = project

In [151]:
# Use local data file or download from some type of bucket
import os

data_path=os.path.join(server_path,data_file)
data_path

'Users/newzysharma/Desktop/Desktop/Machine_Learning/Project/hour.csv'

In [152]:
# assign target and inputs for logistic regression
y = target
X = [name for name in hp.columns if name != y]
print(y)
print(X)

cnt
['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered']


In [153]:
# impute missing values
_ = hp[reals].impute(method='mean')
_ = hp[ints].impute(method='median')

if scale:
    hp[reals] = hp[reals].scale()
    hp[ints] = hp[ints].scale()

In [154]:
if classification:
    class_percentage = y_balance=df[y].mean()[0]/(df[y].max()-df[y].min())
    if class_percentage < balance_threshold:
        balance_y=True
        

print(run_time)
type(run_time)

1850


int

## Cross-validate rather than take a test training split with 1850 seconds

In [155]:
# automl
# runs for run_time seconds then builds a stacked ensemble

aml = H2OAutoML(max_runtime_secs=run_time,project_name = project) # init automl, run for 1850 seconds
aml.train(x=X,  
           y=y,
           training_frame=hp)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Leaderboard

In [156]:
# view leaderboard
lb = aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_2_AutoML_20190408_171007,10.3989,3.22474,10.3989,1.88964,0.0353651
GBM_2_AutoML_20190408_194251,12.7928,3.57671,12.7928,2.13437,0.0450689
GBM_3_AutoML_20190408_203115,12.9605,3.60007,12.9605,2.11321,0.0405568
GBM_1_AutoML_20190408_171007,13.2483,3.63982,13.2483,2.30598,0.0573338
GBM_1_AutoML_20190408_203115,13.4612,3.66895,13.4612,2.19063,0.0428744
GBM_4_AutoML_20190408_190707,14.2118,3.76985,14.2118,2.15209,0.0367956
GBM_3_AutoML_20190408_194251,14.2999,3.78152,14.2999,2.17271,0.0358335
XGBoost_1_AutoML_20190408_171007,14.936,3.86471,14.936,2.25298,0.0426858
GBM_3_AutoML_20190408_190707,15.2039,3.89922,15.2039,2.27726,0.0438891
XGBoost_3_AutoML_20190408_194251,15.9838,3.99797,15.9838,2.48441,nan


In [157]:
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_2_AutoML_20190408_171007


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 2.7545139005515074
RMSE: 1.659672829370749
MAE: 1.1420248950973013
RMSLE: 0.026260655890205944
Mean Residual Deviance: 2.7545139005515074

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 10.398929973581447
RMSE: 3.2247371944984056
MAE: 1.8896396840192615
RMSLE: 0.03536506290104928
Mean Residual Deviance: 10.398929973581447
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,1.8896418,0.0331531,1.8888159,1.8440878,1.955546,1.8330745,1.9266849
mean_residual_deviance,10.398917,1.1037041,9.484616,8.705696,13.301646,10.319401,10.1832285
mse,10.398917,1.1037041,9.484616,8.705696,13.301646,10.319401,10.1832285
r2,0.9996837,0.0000345,0.9997121,0.9997374,0.9995934,0.9996870,0.9996888
residual_deviance,10.398917,1.1037041,9.484616,8.705696,13.301646,10.319401,10.1832285
rmse,3.2161787,0.1659996,3.0797105,2.9505417,3.6471422,3.2123823,3.191117
rmsle,0.0353167,0.0013090,0.0337463,0.0375869,0.0355834,0.0327149,0.0369519


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-04-08 17:25:27,16.816 sec,0.0,181.3823804,142.3998489,32899.5679309
,2019-04-08 17:25:27,16.882 sec,5.0,107.4239264,84.2400360,11539.8999535
,2019-04-08 17:25:27,16.935 sec,10.0,63.8940838,50.0233089,4082.4539467
,2019-04-08 17:25:27,16.985 sec,15.0,38.4509700,30.0349704,1478.4770913
,2019-04-08 17:25:27,17.034 sec,20.0,25.6530386,19.7348606,658.0783895
---,---,---,---,---,---,---
,2019-04-08 17:25:30,19.344 sec,270.0,1.7050142,1.1700358,2.9070734
,2019-04-08 17:25:30,19.393 sec,275.0,1.6909674,1.1622558,2.8593708
,2019-04-08 17:25:30,19.451 sec,280.0,1.6770692,1.1544659,2.8125612
,2019-04-08 17:25:30,19.520 sec,285.0,1.6617166,1.1441284,2.7613021



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
registered,2568583680.0000000,1.0,0.9182406
casual,153480320.0000000,0.0597529,0.0548675
hr,43721364.0000000,0.0170216,0.0156299
instant,9335018.0,0.0036343,0.0033372
workingday,7972549.0,0.0031039,0.0028501
dteday,5198927.5,0.0020240,0.0018586
temp,3354083.5,0.0013058,0.0011990
weekday,2513148.2500000,0.0009784,0.0008984
atemp,2094532.2500000,0.0008154,0.0007488
hum,538358.8125000,0.0002096,0.0001925


In [158]:
aml.leader.algo

'gbm'

## Ensemble Exploration

In [159]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,GBM_2_AutoML_20190408_171007,10.398930,3.224737,10.398930,1.889640,0.035365
1,GBM_2_AutoML_20190408_194251,12.792848,3.576709,12.792848,2.134374,0.045069
2,GBM_3_AutoML_20190408_203115,12.960531,3.600074,12.960531,2.113213,0.040557
3,GBM_1_AutoML_20190408_171007,13.248308,3.639823,13.248308,2.305975,0.057334
4,GBM_1_AutoML_20190408_203115,13.461223,3.668954,13.461223,2.190633,0.042874
5,GBM_4_AutoML_20190408_190707,14.211758,3.769849,14.211758,2.152092,0.036796
6,GBM_3_AutoML_20190408_194251,14.299879,3.781518,14.299879,2.172706,0.035834
7,XGBoost_1_AutoML_20190408_171007,14.936004,3.864713,14.936004,2.252979,0.042686
8,GBM_3_AutoML_20190408_190707,15.203899,3.899218,15.203899,2.277258,0.043889
9,XGBoost_3_AutoML_20190408_194251,15.983771,3.997971,15.983771,2.484415,NaN


## Getting Models

### Parameters for GBM_2_AutoML_20190408_171007

In [160]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [161]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_2_AutoML_20190408_171007',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_2_AutoML_20190408_171007'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 5},
 'fold_assignment': {'default': 'AUTO',

### Parameters for XGBoost_1_AutoML_20190408_190707

In [162]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[17])

In [163]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'XGBoost_1_AutoML_20190408_190707',
   'type': 'Key<Model>',
   'URL': '/3/Models/XGBoost_1_AutoML_20190408_190707'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'fold_assignment': {'default': 'AUTO', 'actual': 'Modulo'},
 'fold_column': {'defau

### Parameters for StackedEnsemble_AllModels_AutoML_20190408_171007

In [164]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[20])

In [165]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'StackedEnsemble_AllModels_AutoML_20190408_171007',
   'type': 'Key<Model>',
   'URL': '/3/Models/StackedEnsemble_AllModels_AutoML_20190408_171007'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'response_column': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ColSpecifierV3',
    'schema_type': 'VecSpecifier'},
   'column_name': 'cnt',
   'is_member_of_frames': None}},
 'validation_frame': {'default': None, 'actual': None},
 'blending_frame': {'default': None, 'actual': None},
 'base_models': {'default': [],
  'actual': [{'__meta': {'schema_version': 3,
     'schema_name': 'ModelKey

# CONCLUSION

<table style="width:50%">
  <tr>
      <th>Runtime of model in Number of Seconds<br></th>
    <th>Models Generated</th> 
  </tr>
    
   <tr>
    <td>500</td>
    <td>7</td> 
  </tr>
    
  <tr>
    <td>1000</td>
    <td>13</td> 
  </tr>
  
  <tr>
    <td>1350</td>
    <td>33</td> 
  </tr>
  
   <tr>
    <td>1500</td>
    <td>56</td> 
  </tr>
  
   <tr>
    <td>1850</td>
    <td>84<td> 
  </tr>
    
</table>

# CONTRIBUTION

# CITATIONS

https://github.com/nikbearbrown/CSYE_7245/blob/master/H2O/H2O_automl_model.ipynb

# LICENSE


Copyright 2019 Newzy Sharma 

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated 
documentation files (the "Software"), to deal in the Software without restriction, including without limitation the 
rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit
persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the 
Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE 
WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR 
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR 
OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.